# Calculating Forest Loss at the Ethnologue Polygon Level

In [15]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [16]:
# IMPORTING DATA 

# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
ethnologue = ee.FeatureCollection('projects/ee-juamiji/assets/langa_no_overlap_biggest_clean')

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

In [17]:
# PREPARING THE DATA

# Select the tree cover and loss bands, and filter for tree cover greater than 30% in 2000
treecover00=flossHansen.select('treecover2000').gte(30)
treeloss=flossHansen.select('loss')

In [18]:
# DEFINING A FUNCTION TO CALCULATE AREA

def calculate_area(feature):
    # Multiply treecover and loss by pixel area to get square meters
    treecover_area_img = treecover00.multiply(ee.Image.pixelArea())
    treeloss_area_img = treeloss.multiply(ee.Image.pixelArea())

    # Sum over the feature's geometry
    treecover_area = treecover_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13,
        bestEffort=True
    ).get('treecover2000')  
    
    loss_area = treeloss_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13,
        bestEffort=True
    ).get('loss')  

    # Return feature with new properties
    return feature.set({
        'forestcover00': treecover_area,
        'forestloss': loss_area
    })

# Apply the function to each polygon
ethnologue_forest = ethnologue.map(calculate_area)


In [19]:
# Export to shapefile
task = ee.batch.Export.table.toDrive(
    collection=ethnologue_forest,
    description='ethnologue_forest_shapefile',
    folder='EarthEngineExports',
    fileFormat='SHP'
)

task.start()
print("Exporting shapefile to Google Drive...")

Exporting shapefile to Google Drive...


In [9]:
# Checking the results
sample = ethnologue_forest.limit(10)
features = sample.getInfo()['features']
for f in features:
    print(f['properties'])

{'ID': 'AUJ-LBY', 'forestcover00': 0}
{'ID': 'SWN-LBY', 'forestcover00': 0}
{'ID': 'SIZ-EGY', 'forestcover00': 0}
{'ID': 'CNU-DZA', 'forestcover00': 104502.408108341}
{'ID': 'TZM-MAR', 'forestcover00': 2419983763.55267}
{'ID': 'SHI-MAR', 'forestcover00': 353911948.01575214}
{'ID': 'TZM-DZA', 'forestcover00': 0}
{'ID': 'SHI-DZA', 'forestcover00': 0}
{'ID': 'KAB-DZA', 'forestcover00': 3501564289.6009374}
{'ID': 'JBN-LBY', 'forestcover00': 0}


In [11]:
# Keep only features where forestcover00 is greater than 0
ethnologue_forest_no0 = ethnologue_forest.filter(ee.Filter.gt('forestcover00', 0))

In [12]:
# EXPORTING THE RESULTS

task = ee.batch.Export.table.toDrive(
    collection=ethnologue_forest_no0,
    description='ethnologue_forest_export',
    folder="EarthEngineExports",
    fileFormat='CSV'  # or 'GeoJSON'
)

task.start()
print(f"Exporting to Google Drive...")

Exporting to Google Drive...


In [14]:
out_dir = os.path.expanduser("~/Downloads")

### NOTE: see the progress of exports at [GEE Task Manager](https://code.earthengine.google.com/tasks)